# Análisis del Campeonato

El siguiente notebook genera un archivo `.xlsx` con las posiciones del campeonato codificadas con colores, gráficos de posiciones por equipo y gráficos de posiciones de todos los equipos


## Setup

Primero hay que parsear las variables $\beta_{il}$

In [6]:
beta_m_init, beta_p_init = {}, {}
beta_m_opt, beta_p_opt = {}, {}

In [7]:
base_dir = '../logs/solutions'
champ = 'grecia'
file_1 = f'model-{champ}-init-0'
file_2 = f'model-{champ}-opt-*'

In [8]:
with open(f'{base_dir}/{champ}/{file_1}.sol', 'r', encoding='utf-8') as infile:
    for line in infile:
        line = line.strip()
        if 'beta' in line:
            var, value = line.split()
            _, var = var.split('[')
            var = var.strip(']')
            team, date = var.split(',')
            if 'beta_p' in line:
                beta_p_init[team, int(date)] = round(float(value))
            else:
                beta_m_init[team, int(date)] = round(float(value))

In [9]:
with open(f'{base_dir}/{champ}/{file_2}.sol', 'r', encoding='utf-8') as infile:
    for line in infile:
        line = line.strip()
        if 'beta' in line:
            var, value = line.split()
            _, var = var.split('[')
            var = var.strip(']')
            team, date = var.split(',')
            if 'beta_p' in line:
                beta_p_opt[team, int(date)] = round(float(value))
            else:
                beta_m_opt[team, int(date)] = round(float(value))

Obtenemos equipos y fechas

In [10]:
teams = list({i[0] for i in beta_m_init.keys()})
teams.sort()
dates = list({i[1] for i in beta_m_init.keys()})

In [11]:
teams

['AEK',
 'APO',
 'ASA',
 'AST',
 'ATR',
 'ION',
 'LAM',
 'OFI',
 'OLY',
 'PAN',
 'PAO',
 'PAS',
 'PNT',
 'VOL']

Finalmente se construye una matriz, donde las filas son los equipos, las columnas las fechas y los valores de las celdas corresponden a la mejor y peor posición que puede alcnazar el equipo

In [12]:
m_init, m_opt = [], []
for team in teams:
    t_init, t_opt = [], []
    for date in dates:
        t_init.append((beta_m_init[team, date], beta_p_init[team,date]))
        t_opt.append((beta_m_opt[team, date], beta_p_opt[team,date]))
    m_init.append(t_init)
    m_opt.append(t_opt)

## Write excel

In [13]:
 import xlsxwriter

 # Create a workbook and add a worksheet.
 workbook = xlsxwriter.Workbook(f'resultados_{file_1}.xlsx')
 worksheet = workbook.add_worksheet()

In [14]:
red = workbook.add_format({'bg_color': 'red'})
green = workbook.add_format({'bg_color': 'green'})
yellow = workbook.add_format({'bg_color': 'yellow'})

In [15]:
for i, date in enumerate(dates):
    worksheet.write(0, i + 1, date)

In [16]:
for i, team in enumerate(teams):
    worksheet.write(i + 1, 0, team)

In [17]:
for i, team in enumerate(teams):
    for j, results in enumerate(m_init[i]):
        val = "/".join([str(i) for i in results])
        if results[0] == 1 and results[1] == len(teams):
            cell = green
        elif results[0] == 1 or results[1] == len(teams):
            cell = yellow
        else:
            cell = red
        worksheet.write(i + 1, j + 1, val, cell)

In [18]:
 workbook.close()

In [19]:
import xlsxwriter

# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook(f'resultados_{file_2}.xlsx')
worksheet = workbook.add_worksheet()

red = workbook.add_format({'bg_color': 'red'})
green = workbook.add_format({'bg_color': 'green'})
yellow = workbook.add_format({'bg_color': 'yellow'})

In [20]:
for i, date in enumerate(dates):
    worksheet.write(0, i + 1, date)

for i, team in enumerate(teams):
    worksheet.write(i + 1, 0, team)

In [21]:
for i, team in enumerate(teams):
    for j, results in enumerate(m_opt[i]):
        val = "/".join([str(i) for i in results])
        if results[0] == 1 and results[1] == len(teams):
            cell = green
        elif results[0] == 1 or results[1] == len(teams):
            cell = yellow
        else:
            cell = red
        worksheet.write(i + 1, j + 1, val, cell)

In [22]:
 workbook.close()

## Grafico de mejor y peor caso para equipos

In [20]:
team = 'CGC'

In [21]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [22]:
best_case_init = [beta_m_init[team, date] for date in dates]
worst_case_init = [beta_p_init[team, date] for date in dates]
best_case_opt = [beta_m_opt[team, date] for date in dates]
worst_case_opt = [beta_p_opt[team, date] for date in dates]

In [23]:
plt.plot(best_case_init[:-1], linestyle = 'dotted')
plt.plot(worst_case_init[:-1], linestyle = 'dotted')

plt.title(f"{team} Inicial")

plt.xlabel("Fecha")
plt.ylabel("Posición")

plt.grid(linestyle = ':', linewidth = 0.5)
plt.legend(['Mejor caso', 'Peor caso'])

plt.show()

NameError: name 'plt' is not defined

In [24]:
plt.plot(best_case_opt[:-1], linestyle = 'dotted')
plt.plot(worst_case_opt[:-1], linestyle = 'dotted')

plt.title(f"{team} Optimizado")

plt.xlabel("Fecha")
plt.ylabel("Posición")

plt.grid(linestyle = ':', linewidth = 0.5)
plt.legend(['Mejor caso', 'Peor caso'])

plt.show()

NameError: name 'plt' is not defined

## Card at k

Para una fecha, contar la cantidad de equipos que pueden llegar a la k mejor y peor posición.

In [25]:
def get_card_best(beta, date, k=1):
    count = 0
    for team in teams:
        pos = beta[team, date]
        if pos <= k:
            count += 1
    return count

In [26]:
def get_card_worst(beta, date, k=1):
    count = 0
    for team in teams:
        pos = beta[team, date]
        if pos >= len(teams) + 1 - k:
            count += 1
    return count

In [27]:
card_best_1_init = [get_card_best(beta_m_init, date, 1) for date in dates]
card_worst_1_init = [get_card_worst(beta_p_init, date, 1) for date in dates]
card_best_1_opt = [get_card_best(beta_m_opt, date, 1) for date in dates]
card_worst_1_opt = [get_card_worst(beta_p_opt, date, 1) for date in dates]

In [28]:
plt.plot(card_best_1_init[:-1], linestyle = 'dotted')
plt.plot(card_best_1_opt[:-1], linestyle = 'dotted')

plt.title('Cantidad de equipos que pueden salir 1')

plt.xlabel("Fecha")
plt.ylabel("Equipos")

plt.grid(linestyle = ':', linewidth = 0.5)
plt.legend(['Original', 'Optimizado'])

plt.show()

NameError: name 'plt' is not defined

In [29]:
plt.plot(card_best_1_opt, linestyle = 'dotted')
plt.plot(card_worst_1_opt, linestyle = 'dotted')

plt.title('Cantidad de equipos que pueden salir primero o últimos')

plt.xlabel("Fecha")
plt.ylabel("Equipos")

plt.grid(linestyle = ':', linewidth = 0.5)
plt.legend(['Primero', 'Ultimo'])

plt.show()

NameError: name 'plt' is not defined